<a href="https://colab.research.google.com/github/DesmondChoy/llm_tutorials/blob/main/Instructor_Solve_simpler_subproblems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install instructor pydantic google-generativeai -qq

In [2]:
import instructor
import os

from pydantic import BaseModel
import google.generativeai as genai
# from openai import OpenAI
from typing import Iterable, Union
from google.colab import userdata


# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))



In [3]:
class Subquestion(BaseModel):
    question: str


class Answer(BaseModel):
    answer: Union[str, int]


class SubquestionWithAnswers(BaseModel):
    question: str
    answer: Union[str, int]


In [4]:
# client = instructor.from_openai(OpenAI())

client = instructor.from_gemini(
    client=genai.GenerativeModel(
        model_name="models/gemini-2.0-flash-exp",
    ),
    mode=instructor.Mode.GEMINI_JSON,
)

In [5]:
def decompose(question):
    return client.chat.completions.create(
        response_model=Iterable[Subquestion],
        messages=[
            {
                "role": "user",
                "content": f"For this question: {question} - Break it down into simpler subquestions that can be solved sequentially",
            }
        ],
    )

In [6]:
def solve(question, solved_questions, original_question):
    return client.chat.completions.create(
        response_model=Answer,
        messages=[
            {
                "role": "user",
                "content": f"""
                    <original_question>
                    {original_question}
                    </original_question>

                    <solved_subquestions>
                    {solved_questions}
                    </solved_subquestions>

                    Solve this next subquestion: {question}
                    """,
            }
        ],
    ).answer


In [7]:
question = f"""You are standing outside a room with a closed door. There are three light switches on the wall outside the room. Each switch controls a different light bulb inside the room. You cannot see into the room and you cannot open the door until you know which switch controls which bulb.

You are allowed to flip the switches however you like, but you can only enter the room once. After entering the room, you must be able to correctly identify which switch controls which bulb.

?"""

# Stage 1: Decompose Question into Subquestions
subquestions = decompose(question)
subquestions

[Subquestion(question='How can we manipulate the light switches outside the room to gather information about which switch corresponds to which bulb?'),
 Subquestion(question='What are the observable differences when the lights are on or off, besides their illumination?'),
 Subquestion(question='How can we use the heat generated by the bulbs to differentiate between them?'),
 Subquestion(question='Given the manipulations and the observations, how can we match each switch to its corresponding bulb?')]

In [8]:
# Stage 2: Sequentially Solve Subquestions
solved_questions = []
for subquestion in subquestions:
    solved_questions.append(
        SubquestionWithAnswers(
            question=subquestion.question,
            answer=solve(subquestion, solved_questions, question),
        )
    )

# Print
for item in solved_questions:
    print(f"{item.question} {item.answer}")

How can we manipulate the light switches outside the room to gather information about which switch corresponds to which bulb? Flip the first switch and leave it on for a few minutes. Then, turn it off and flip the second switch on. Leave the third switch off. Now you can open the door.
What are the observable differences when the lights are on or off, besides their illumination? The light bulbs that were on for a period of time will be warm to the touch, compared to those that were off or only recently turned on.
How can we use the heat generated by the bulbs to differentiate between them? Enter the room and touch each of the bulbs. The bulb that is on corresponds to the second switch that was left on. The bulb that is warm is from the first switch. The remaining bulb corresponds to the remaining switch.
Given the manipulations and the observations, how can we match each switch to its corresponding bulb? The first switch corresponds to the bulb that is warm. The second switch correspon